In [ ]:
!pip install langchain openai python-dotenv
!pip install langchain_openai

In [ ]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print("🔧 Setting up Azure OpenAI credentials...")
print("You can find these in your Azure OpenAI resource in the Azure portal")

if not os.getenv('AZURE_OPENAI_API_KEY'):
    os.environ['AZURE_OPENAI_API_KEY'] = getpass("Enter your Azure OpenAI API Key: ")

if not os.getenv('AZURE_OPENAI_ENDPOINT'):
    os.environ['AZURE_OPENAI_ENDPOINT'] = input("Enter your Azure OpenAI Endpoint (e.g., https://your-resource.openai.azure.com/): ")

if not os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'):
    deployment_name = input("Enter your GPT-4o deployment name (default: gpt4o): ") or "gpt-4o"
    os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'] = deployment_name

print("✅ Credentials configured!")

🔧 Setting up Azure OpenAI credentials...
You can find these in your Azure OpenAI resource in the Azure portal
Enter your Azure OpenAI API Key: ··········
Enter your Azure OpenAI Endpoint (e.g., https://your-resource.openai.azure.com/): https://chait-ma9dwmvx-eastus.openai.azure.com/
Enter your GPT-4o deployment name (default: gpt4o): gpt-4o
✅ Credentials configured!


In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage

In [ ]:
print("🤖 Initializing Azure OpenAI connection...")

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2024-12-01-preview",
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    temperature=0.7
)


🤖 Initializing Azure OpenAI connection...


In [ ]:
print("🧪 Testing connection...")
test_response = llm.invoke([HumanMessage(content="Say 'Hello from Azure OpenAI!'")])
print(f"✅ Connection successful: {test_response.content}")

🧪 Testing connection...
✅ Connection successful: Hello from Azure OpenAI! 😊


In [ ]:
idea_template = """
You are a creative brainstorming assistant. Generate 5 innovative ideas for the following topic.
Make each idea unique and practical.

Topic: {topic}

Ideas:
"""

In [ ]:
idea_prompt = PromptTemplate(
    input_variables=["topic"],
    template=idea_template
)

In [ ]:
# Create the chain
idea_chain = LLMChain(
    llm=llm,
    prompt=idea_prompt,
    verbose=True  # This shows the prompt being sent
)

/tmp/ipython-input-819140694.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  idea_chain = LLMChain(


In [ ]:
print("🚀 Testing your first chain...")
result = idea_chain.run(topic="sustainable energy solutions")
print("\n" + "="*50)
print("RESULT:")
print(result)

🚀 Testing your first chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a creative brainstorming assistant. Generate 5 innovative ideas for the following topic.
Make each idea unique and practical.

Topic: sustainable energy solutions

Ideas:



/tmp/ipython-input-678142800.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = idea_chain.run(topic="sustainable energy solutions")



> Finished chain.

RESULT:
1. **Solar-Powered Modular Microgrids for Remote Communities**  
   Develop portable, modular microgrid systems powered by solar panels that can be assembled like building blocks. These microgrids could provide sustainable energy to remote, off-grid areas or disaster-stricken regions. The modular design allows for scalability, so communities can expand their energy capacity as needed. The system would include integrated battery storage and could be connected to nearby grids once available.

2. **Kinetic Energy Harvesting Sidewalks**  
   Install piezoelectric tiles in high-foot-traffic urban areas like sidewalks, subway stations, and stadium entrances. These tiles convert the pressure and vibrations from footsteps into electricity. The collected energy could power streetlights, public Wi-Fi hotspots, or EV (electric vehicle) charging stations. Additionally, these tiles could be paired with an app that tracks energy generated by users, incentivizing walking w

In [ ]:
# %% [markdown]
# ## 4. Sequential Chaining - The Main Event
#
# Now let's build our content creation pipeline with multiple connected chains.

# %%
from langchain.chains import SimpleSequentialChain

print("🏗️ Building sequential chain pipeline...")

🏗️ Building sequential chain pipeline...


In [ ]:
# Chain 1: Topic to Outline
outline_template = """
You are a content strategist. Create a detailed outline for the topic: {topic}
Make it structured with main points and sub-points.
Format as a clear, numbered outline.

Topic: {topic}
Outline:"""

outline_prompt = PromptTemplate(
    input_variables=["topic"],
    template=outline_template
)
outline_chain = LLMChain(llm=llm, prompt=outline_prompt, verbose=True)

In [ ]:
# Chain 2: Outline to Content
content_template = """
You are a skilled writer. Using this outline, write detailed, engaging content.
Make it informative and well-structured with proper headings.
Keep it concise but comprehensive.

Outline: {outline}

Content:"""

content_prompt = PromptTemplate(
    input_variables=["outline"],
    template=content_template
)
content_chain = LLMChain(llm=llm, prompt=content_prompt, verbose=True)

In [ ]:
# Chain 3: Content to Summary
summary_template = """
You are an editor. Create a concise summary of this content.
Focus on key takeaways and main points. Keep it under 200 words.

Content: {content}

Summary:"""

summary_prompt = PromptTemplate(
    input_variables=["content"],
    template=summary_template
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, verbose=True)

print("✅ Individual chains created!")

✅ Individual chains created!


# %% [markdown]
# ## 5. Connecting the Chains
#
# Now let's connect all chains into a sequential pipeline.

# %%
# Create the sequential chain

In [ ]:
content_pipeline = SimpleSequentialChain(
    chains=[outline_chain, content_chain, summary_chain],
    verbose=True
)

print("🔗 Sequential chain created!")
print("Pipeline: Topic → Outline → Content → Summary")

🔗 Sequential chain created!
Pipeline: Topic → Outline → Content → Summary


# %% [markdown]
# ## 6. Run the Complete Pipeline
#
# Let's test our content creation pipeline with different topics.

# %%
# Test topics - feel free to modify these!

In [ ]:
test_topics = [
    "Machine Learning for Beginners",
    "Microservices Architecture Best Practices",
    "Azure Kubernetes Service (AKS) Setup"
]

In [ ]:
def run_content_pipeline(topic):
    """Run the content pipeline and display results nicely"""
    print(f"\n{'='*60}")
    print(f"🎯 TOPIC: {topic}")
    print(f"{'='*60}")

    try:
        result = content_pipeline.run(topic)
        print(f"\n✅ FINAL SUMMARY:\n{result}")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Run pipeline for each test topic
for topic in test_topics:
    run_content_pipeline(topic)
    print("\n" + "-"*80 + "\n")

# ## 7. Advanced: Multi-Input Sequential Chain
#
# Let's create a more sophisticated chain that handles multiple inputs and outputs.

# LangChain Sequential Chaining with Azure OpenAI

This notebook demonstrates how to build content creation pipelines using LangChain's sequential chaining with Azure OpenAI.

## 1. Setup and Imports

First, let's install the necessary libraries and import the required modules.

In [ ]:
!pip install langchain openai python-dotenv
!pip install langchain_openai

In [ ]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

## 2. Configure Azure OpenAI Credentials

Enter your Azure OpenAI API key, endpoint, and deployment name.

In [ ]:
print("🔧 Setting up Azure OpenAI credentials...")
print("You can find these in your Azure OpenAI resource in the Azure portal")

if not os.getenv('AZURE_OPENAI_API_KEY'):
    os.environ['AZURE_OPENAI_API_KEY'] = getpass("Enter your Azure OpenAI API Key: ")

if not os.getenv('AZURE_OPENAI_ENDPOINT'):
    os.environ['AZURE_OPENAI_ENDPOINT'] = input("Enter your Azure OpenAI Endpoint (e.g., https://your-resource.openai.azure.com/): ")

if not os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'):
    deployment_name = input("Enter your GPT-4o deployment name (default: gpt4o): ") or "gpt-4o"
    os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'] = deployment_name

print("✅ Credentials configured!")

## 3. Initialize and Test Azure OpenAI Connection

Initialize the AzureChatOpenAI model and test the connection.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage

In [ ]:
print("🤖 Initializing Azure OpenAI connection...")

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2024-12-01-preview",
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    temperature=0.7
)

In [ ]:
print("🧪 Testing connection...")
test_response = llm.invoke([HumanMessage(content="Say 'Hello from Azure OpenAI!'")])
print(f"✅ Connection successful: {test_response.content}")

## 4. Basic Single Chain Example

Let's create a simple chain to brainstorm ideas.

In [ ]:
idea_template = """
You are a creative brainstorming assistant. Generate 5 innovative ideas for the following topic.
Make each idea unique and practical.

Topic: {topic}

Ideas:
"""

In [ ]:
idea_prompt = PromptTemplate(
    input_variables=["topic"],
    template=idea_template
)

In [ ]:
# Create the chain
idea_chain = LLMChain(
    llm=llm,
    prompt=idea_prompt,
    verbose=True  # This shows the prompt being sent
)

In [ ]:
print("🚀 Testing your first chain...")
result = idea_chain.run(topic="sustainable energy solutions")
print("\n" + "="*50)
print("RESULT:")
print(result)

## 5. Sequential Chaining - The Main Event

Now let's build our content creation pipeline with multiple connected chains using `SimpleSequentialChain`.

In [ ]:
from langchain.chains import SimpleSequentialChain

print("🏗️ Building sequential chain pipeline...")

In [ ]:
# Chain 1: Topic to Outline
outline_template = """
You are a content strategist. Create a detailed outline for the topic: {topic}
Make it structured with main points and sub-points.
Format as a clear, numbered outline.

Topic: {topic}
Outline:"""

outline_prompt = PromptTemplate(
    input_variables=["topic"],
    template=outline_template
)
outline_chain = LLMChain(llm=llm, prompt=outline_prompt, verbose=True)

In [ ]:
# Chain 2: Outline to Content
content_template = """
You are a skilled writer. Using this outline, write detailed, engaging content.
Make it informative and well-structured with proper headings.
Keep it concise but comprehensive.

Outline: {outline}

Content:"""

content_prompt = PromptTemplate(
    input_variables=["outline"],
    template=content_template
)
content_chain = LLMChain(llm=llm, prompt=content_prompt, verbose=True)

In [ ]:
# Chain 3: Content to Summary
summary_template = """
You are an editor. Create a concise summary of this content.
Focus on key takeaways and main points. Keep it under 200 words.

Content: {content}

Summary:"""

summary_prompt = PromptTemplate(
    input_variables=["content"],
    template=summary_template
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, verbose=True)

print("✅ Individual chains created!")

## 6. Connecting the Chains

Now let's connect all chains into a sequential pipeline.

In [ ]:
# Create the sequential chain
content_pipeline = SimpleSequentialChain(
    chains=[outline_chain, content_chain, summary_chain],
    verbose=True
)

print("🔗 Sequential chain created!")
print("Pipeline: Topic → Outline → Content → Summary")

## 7. Run the Complete Pipeline

Let's test our content creation pipeline with different topics.

In [ ]:
# Test topics - feel free to modify these!
test_topics = [
    "Machine Learning for Beginners",
    "Microservices Architecture Best Practices",
    "Azure Kubernetes Service (AKS) Setup"
]

In [ ]:
def run_content_pipeline(topic):
    """Run the content pipeline and display results nicely"""
    print(f"\n{'='*60}")
    print(f"🎯 TOPIC: {topic}")
    print(f"{'='*60}")

    try:
        result = content_pipeline.run(topic)
        print(f"\n✅ FINAL SUMMARY:\n{result}")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Run pipeline for each test topic
for topic in test_topics:
    run_content_pipeline(topic)
    print("\n" + "-"*80 + "\n")

## 8. Advanced: Multi-Input Sequential Chain

Let's create a more sophisticated chain that handles multiple inputs and outputs using `SequentialChain`.

In [ ]:
from langchain.chains import SequentialChain

print("🧪 Creating advanced multi-input sequential chain...")

# Enhanced outline chain with audience consideration
advanced_outline_template = """
You are a content strategist. Create a detailed outline for the topic: {topic}
Target audience: {audience}
Writing tone: {tone}

Make it structured and appropriate for your audience.

Topic: {topic}
Audience: {audience}
Tone: {tone}

Outline:"""

advanced_outline_prompt = PromptTemplate(
    input_variables=["topic", "audience", "tone"],
    template=advanced_outline_template
)
advanced_outline_chain = LLMChain(
    llm=llm,
    prompt=advanced_outline_prompt,
    output_key="outline"
)

# Enhanced content chain
advanced_content_template = """
Write detailed content based on this outline.
Target audience: {audience}
Writing tone: {tone}

Tailor your language and examples to the audience.

Outline: {outline}
Audience: {audience}
Tone: {tone}

Content:"""

advanced_content_prompt = PromptTemplate(
    input_variables=["outline", "audience", "tone"],
    template=advanced_content_template
)
advanced_content_chain = LLMChain(
    llm=llm,
    prompt=advanced_content_prompt,
    output_key="content"
)

# Enhanced summary chain
advanced_summary_template = """
Create a summary tailored to: {audience}
Use a {tone} tone.

Content: {content}
Audience: {audience}
Tone: {tone}

Summary:"""

advanced_summary_prompt = PromptTemplate(
    input_variables=["content", "audience", "tone"],
    template=advanced_summary_template
)
advanced_summary_chain = LLMChain(
    llm=llm,
    prompt=advanced_summary_prompt,
    output_key="summary"
)

# Create the advanced sequential chain
advanced_pipeline = SequentialChain(
    chains=[advanced_outline_chain, advanced_content_chain, advanced_summary_chain],
    input_variables=["topic", "audience", "tone"],
    output_variables=["outline", "content", "summary"],
    verbose=True
)

print("✅ Advanced sequential chain created!")

## 9. Test the Advanced Pipeline

Run the advanced pipeline with different inputs.

In [ ]:
def run_advanced_pipeline(topic, audience, tone):
    """Run the advanced pipeline with multiple inputs"""
    print(f"\n{'='*60}")
    print(f"🎯 TOPIC: {topic}")
    print(f"👥 AUDIENCE: {audience}")
    print(f"🎭 TONE: {tone}")
    print(f"{'='*60}")

    try:
        result = advanced_pipeline({
            "topic": topic,
            "audience": audience,
            "tone": tone
        })

        print(f"\n📋 OUTLINE:\n{result['outline'][:200]}...")
        print(f"\n📝 CONTENT:\n{result['content'][:300]}...")
        print(f"\n✅ SUMMARY:\n{result['summary']}")

        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Test different combinations
test_cases = [
    {
        "topic": "Kubernetes for Production",
        "audience": "DevOps engineers",
        "tone": "technical and detailed"
    },
    {
        "topic": "Cloud Computing Basics",
        "audience": "business executives",
        "tone": "friendly and non-technical"
    },
    {
        "topic": "API Design Patterns",
        "audience": "junior developers",
        "tone": "educational and encouraging"
    }
]

for test_case in test_cases:
    run_advanced_pipeline(**test_case)
    print("\n" + "-"*80 + "\n")

## 10. Error Handling and Debugging

Let's implement robust error handling for production use.

In [ ]:
import logging
from typing import Dict, Any

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class RobustContentPipeline:
    def __init__(self, llm):
        self.llm = llm
        self.setup_chains()

    def setup_chains(self):
        """Initialize all chains with error handling"""
        try:
            # Outline chain
            self.outline_chain = LLMChain(
                llm=self.llm,
                prompt=PromptTemplate(
                    input_variables=["topic"],
                    template="Create a brief outline for: {topic}\nOutline:"
                )
            )

            # Content chain
            self.content_chain = LLMChain(
                llm=self.llm,
                prompt=PromptTemplate(
                    input_variables=["outline"],
                    template="Write content based on: {outline}\nContent:"
                )
            )

            # Summary chain
            self.summary_chain = LLMChain(
                llm=self.llm,
                prompt=PromptTemplate(
                    input_variables=["content"],
                    template="Summarize: {content}\nSummary:"
                )
            )

            logger.info("✅ All chains initialized successfully")

        except Exception as e:
            logger.error(f"❌ Failed to initialize chains: {e}")
            raise

    def run_pipeline(self, topic: str) -> Dict[str, Any]:
        """Run the complete pipeline with error handling"""
        results = {"topic": topic}

        try:
            # Step 1: Generate outline
            logger.info("🔄 Generating outline...")
            outline = self.outline_chain.run(topic=topic)
            results["outline"] = outline
            logger.info("✅ Outline generated")

            # Step 2: Generate content
            logger.info("🔄 Generating content...")
            content = self.content_chain.run(outline=outline)
            results["content"] = content
            logger.info("✅ Content generated")

            # Step 3: Generate summary
            logger.info("🔄 Generating summary...")
            summary = self.summary_chain.run(content=content)
            results["summary"] = summary
            logger.info("✅ Summary generated")

            results["status"] = "success"

        except Exception as e:
            logger.error(f"❌ Pipeline failed at step: {e}")
            results["status"] = "failed"
            results["error"] = str(e)

        return results

# Test the robust pipeline
print("🛡️ Testing robust pipeline with error handling...")

robust_pipeline = RobustContentPipeline(llm)
result = robust_pipeline.run_pipeline("Azure Container Instances vs Azure Kubernetes Service")

print("\n" + "="*50)
print("ROBUST PIPELINE RESULT:")
print(f"Status: {result['status']}")
if result['status'] == 'success':
    print(f"Summary: {result['summary']}")
else:
    print(f"Error: {result.get('error', 'Unknown error')}")

## 11. Interactive Testing Section

Now you can test with your own topics and configurations!

In [ ]:
# Interactive testing function
def interactive_test():
    """Interactive function for testing different topics"""
    print("🎮 Interactive Testing Mode")
    print("Enter 'quit' to exit")

    while True:
        topic = input("\nEnter a topic to generate content for: ")
        if topic.lower() == 'quit':
            print("👋 Goodbye!")
            break

        if topic.strip():
            print(f"\n🚀 Processing: {topic}")
            result = robust_pipeline.run_pipeline(topic)

            if result['status'] == 'success':
                print(f"\n✅ SUMMARY:\n{result['summary']}")
            else:
                print(f"\n❌ ERROR: {result.get('error', 'Unknown error')}")
        else:
            print("⚠️ Please enter a valid topic")

# Uncomment the line below to start interactive mode
# interactive_test()

## 12. Performance Monitoring

Let's add some basic performance monitoring to understand execution times.

In [ ]:
import time
from typing import List

class PerformanceMonitor:
    def __init__(self):
        self.metrics = []

    def time_execution(self, func, *args, **kwargs):
        """Time the execution of a function"""
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()

        execution_time = end_time - start_time
        self.metrics.append({
            'function': func.__name__,
            'execution_time': execution_time,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
        })

        return result, execution_time

    def get_average_time(self) -> float:
        """Get average execution time"""
        if not self.metrics:
            return 0
        return sum(m['execution_time'] for m in self.metrics) / len(self.metrics)

    def print_stats(self):
        """Print performance statistics"""
        if not self.metrics:
            print("📊 No metrics available")
            return

        print("📊 PERFORMANCE STATS:")
        print(f"Total executions: {len(self.metrics)}")
        print(f"Average time: {self.get_average_time():.2f} seconds")
        print(f"Fastest: {min(m['execution_time'] for m in self.metrics):.2f}s")
        print(f"Slowest: {max(m['execution_time'] for m in self.metrics):.2f}s")

# Test performance monitoring
monitor = PerformanceMonitor()

def timed_pipeline_run(topic):
    """Wrapper function for timing pipeline execution"""
    return robust_pipeline.run_pipeline(topic)

# Run multiple tests to gather performance data
test_topics = [
    "Docker containerization best practices",
    "RESTful API design principles",
    "Database indexing strategies"
]

print("⏱️ Running performance tests...")

for topic in test_topics:
    print(f"\n🔄 Testing: {topic}")
    result, exec_time = monitor.time_execution(timed_pipeline_run, topic)
    print(f"⏰ Execution time: {exec_time:.2f} seconds")

# Display performance statistics
print("\n" + "="*50)
monitor.print_stats()

## 13. Key Takeaways and Next Steps

Congratulations! You've learned the core concepts of LangChain sequential chaining.

### What You've Accomplished:
- ✅ Set up Azure OpenAI with LangChain
- ✅ Created basic and advanced sequential chains
- ✅ Implemented error handling and monitoring
- ✅ Built a production-ready content pipeline

### Next Steps to Explore:
1. **Memory Integration** - Add conversation history to chains
2. **Router Chains** - Conditional logic based on inputs
3. **Map-Reduce Chains** - Process large datasets in parallel
4. **Custom Chains** - Build domain-specific logic
5. **Agent Frameworks** - Let LLMs use tools and make decisions

### Integration with Your Stack:
- Deploy chains as **Azure Functions**
- Use with **Azure Container Instances** or **AKS**
- Integrate with your **.NET Core APIs**
- Monitor with **Application Insights** and **Grafana**

In [ ]:
print("🎉 Tutorial Complete!")
print("\nYou now have a solid foundation in LangChain sequential chaining.")
print("Feel free to modify the code above and experiment with different:")
print("- Topics and prompts")
print("- Chain configurations")
print("- Error handling strategies")
print("- Performance optimizations")
print("\nHappy coding! 🚀")

## Bonus: Quick Reference

In [ ]:
from langchain.chains import SequentialChain

print("🧪 Creating advanced multi-input sequential chain...")

# Enhanced outline chain with audience consideration
advanced_outline_template = """
You are a content strategist. Create a detailed outline for the topic: {topic}
Target audience: {audience}
Writing tone: {tone}

Make it structured and appropriate for your audience.

Topic: {topic}
Audience: {audience}
Tone: {tone}

Outline:"""

advanced_outline_prompt = PromptTemplate(
    input_variables=["topic", "audience", "tone"],
    template=advanced_outline_template
)
advanced_outline_chain = LLMChain(
    llm=llm,
    prompt=advanced_outline_prompt,
    output_key="outline"
)

# Enhanced content chain
advanced_content_template = """
Write detailed content based on this outline.
Target audience: {audience}
Writing tone: {tone}

Tailor your language and examples to the audience.

Outline: {outline}
Audience: {audience}
Tone: {tone}

Content:"""

advanced_content_prompt = PromptTemplate(
    input_variables=["outline", "audience", "tone"],
    template=advanced_content_template
)
advanced_content_chain = LLMChain(
    llm=llm,
    prompt=advanced_content_prompt,
    output_key="content"
)

# Enhanced summary chain
advanced_summary_template = """
Create a summary tailored to: {audience}
Use a {tone} tone.

Content: {content}
Audience: {audience}
Tone: {tone}

Summary:"""

advanced_summary_prompt = PromptTemplate(
    input_variables=["content", "audience", "tone"],
    template=advanced_summary_template
)
advanced_summary_chain = LLMChain(
    llm=llm,
    prompt=advanced_summary_prompt,
    output_key="summary"
)

# Create the advanced sequential chain
advanced_pipeline = SequentialChain(
    chains=[advanced_outline_chain, advanced_content_chain, advanced_summary_chain],
    input_variables=["topic", "audience", "tone"],
    output_variables=["outline", "content", "summary"],
    verbose=True
)

print("✅ Advanced sequential chain created!")


# ## 8. Test the Advanced Pipeline

def run_advanced_pipeline(topic, audience, tone):
    """Run the advanced pipeline with multiple inputs"""
    print(f"\n{'='*60}")
    print(f"🎯 TOPIC: {topic}")
    print(f"👥 AUDIENCE: {audience}")
    print(f"🎭 TONE: {tone}")
    print(f"{'='*60}")

    try:
        result = advanced_pipeline({
            "topic": topic,
            "audience": audience,
            "tone": tone
        })

        print(f"\n📋 OUTLINE:\n{result['outline'][:200]}...")
        print(f"\n📝 CONTENT:\n{result['content'][:300]}...")
        print(f"\n✅ SUMMARY:\n{result['summary']}")

        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Test different combinations
test_cases = [
    {
        "topic": "Kubernetes for Production",
        "audience": "DevOps engineers",
        "tone": "technical and detailed"
    },
    {
        "topic": "Cloud Computing Basics",
        "audience": "business executives",
        "tone": "friendly and non-technical"
    },
    {
        "topic": "API Design Patterns",
        "audience": "junior developers",
        "tone": "educational and encouraging"
    }
]

for test_case in test_cases:
    run_advanced_pipeline(**test_case)
    print("\n" + "-"*80 + "\n")

#
# ## 9. Error Handling and Debugging
#
# Let's implement robust error handling for production use.

#
import logging
from typing import Dict, Any

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class RobustContentPipeline:
    def __init__(self, llm):
        self.llm = llm
        self.setup_chains()

    def setup_chains(self):
        """Initialize all chains with error handling"""
        try:
            # Outline chain
            self.outline_chain = LLMChain(
                llm=self.llm,
                prompt=PromptTemplate(
                    input_variables=["topic"],
                    template="Create a brief outline for: {topic}\nOutline:"
                )
            )

            # Content chain
            self.content_chain = LLMChain(
                llm=self.llm,
                prompt=PromptTemplate(
                    input_variables=["outline"],
                    template="Write content based on: {outline}\nContent:"
                )
            )

            # Summary chain
            self.summary_chain = LLMChain(
                llm=self.llm,
                prompt=PromptTemplate(
                    input_variables=["content"],
                    template="Summarize: {content}\nSummary:"
                )
            )

            logger.info("✅ All chains initialized successfully")

        except Exception as e:
            logger.error(f"❌ Failed to initialize chains: {e}")
            raise

    def run_pipeline(self, topic: str) -> Dict[str, Any]:
        """Run the complete pipeline with error handling"""
        results = {"topic": topic}

        try:
            # Step 1: Generate outline
            logger.info("🔄 Generating outline...")
            outline = self.outline_chain.run(topic=topic)
            results["outline"] = outline
            logger.info("✅ Outline generated")

            # Step 2: Generate content
            logger.info("🔄 Generating content...")
            content = self.content_chain.run(outline=outline)
            results["content"] = content
            logger.info("✅ Content generated")

            # Step 3: Generate summary
            logger.info("🔄 Generating summary...")
            summary = self.summary_chain.run(content=content)
            results["summary"] = summary
            logger.info("✅ Summary generated")

            results["status"] = "success"

        except Exception as e:
            logger.error(f"❌ Pipeline failed at step: {e}")
            results["status"] = "failed"
            results["error"] = str(e)

        return results

# Test the robust pipeline
print("🛡️ Testing robust pipeline with error handling...")

robust_pipeline = RobustContentPipeline(llm)
result = robust_pipeline.run_pipeline("Azure Container Instances vs Azure Kubernetes Service")

print("\n" + "="*50)
print("ROBUST PIPELINE RESULT:")
print(f"Status: {result['status']}")
if result['status'] == 'success':
    print(f"Summary: {result['summary']}")
else:
    print(f"Error: {result.get('error', 'Unknown error')}")

#
# ## 10. Interactive Testing Section
#
# Now you can test with your own topics and configurations!

# %%
# Interactive testing function
def interactive_test():
    """Interactive function for testing different topics"""
    print("🎮 Interactive Testing Mode")
    print("Enter 'quit' to exit")

    while True:
        topic = input("\nEnter a topic to generate content for: ")
        if topic.lower() == 'quit':
            print("👋 Goodbye!")
            break

        if topic.strip():
            print(f"\n🚀 Processing: {topic}")
            result = robust_pipeline.run_pipeline(topic)

            if result['status'] == 'success':
                print(f"\n✅ SUMMARY:\n{result['summary']}")
            else:
                print(f"\n❌ ERROR: {result.get('error', 'Unknown error')}")
        else:
            print("⚠️ Please enter a valid topic")

# Uncomment the line below to start interactive mode
# interactive_test()

#
# ## 11. Performance Monitoring
#
# Let's add some basic performance monitoring to understand execution times.

# %%
import time
from typing import List

class PerformanceMonitor:
    def __init__(self):
        self.metrics = []

    def time_execution(self, func, *args, **kwargs):
        """Time the execution of a function"""
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()

        execution_time = end_time - start_time
        self.metrics.append({
            'function': func.__name__,
            'execution_time': execution_time,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
        })

        return result, execution_time

    def get_average_time(self) -> float:
        """Get average execution time"""
        if not self.metrics:
            return 0
        return sum(m['execution_time'] for m in self.metrics) / len(self.metrics)

    def print_stats(self):
        """Print performance statistics"""
        if not self.metrics:
            print("📊 No metrics available")
            return

        print("📊 PERFORMANCE STATS:")
        print(f"Total executions: {len(self.metrics)}")
        print(f"Average time: {self.get_average_time():.2f} seconds")
        print(f"Fastest: {min(m['execution_time'] for m in self.metrics):.2f}s")
        print(f"Slowest: {max(m['execution_time'] for m in self.metrics):.2f}s")

# Test performance monitoring
monitor = PerformanceMonitor()

def timed_pipeline_run(topic):
    """Wrapper function for timing pipeline execution"""
    return robust_pipeline.run_pipeline(topic)

# Run multiple tests to gather performance data
test_topics = [
    "Docker containerization best practices",
    "RESTful API design principles",
    "Database indexing strategies"
]

print("⏱️ Running performance tests...")

for topic in test_topics:
    print(f"\n🔄 Testing: {topic}")
    result, exec_time = monitor.time_execution(timed_pipeline_run, topic)
    print(f"⏰ Execution time: {exec_time:.2f} seconds")

# Display performance statistics
print("\n" + "="*50)
monitor.print_stats()

#
# ## 12. Key Takeaways and Next Steps
#
# Congratulations! You've learned the core concepts of LangChain sequential chaining.
#
# ### What You've Accomplished:
# - ✅ Set up Azure OpenAI with LangChain
# - ✅ Created basic and advanced sequential chains
# - ✅ Implemented error handling and monitoring
# - ✅ Built a production-ready content pipeline
#
# ### Next Steps to Explore:
# 1. **Memory Integration** - Add conversation history to chains
# 2. **Router Chains** - Conditional logic based on inputs
# 3. **Map-Reduce Chains** - Process large datasets in parallel
# 4. **Custom Chains** - Build domain-specific logic
# 5. **Agent Frameworks** - Let LLMs use tools and make decisions
#
# ### Integration with Your Stack:
# - Deploy chains as **Azure Functions**
# - Use with **Azure Container Instances** or **AKS**
# - Integrate with your **.NET Core APIs**
# - Monitor with **Application Insights** and **Grafana**

# %%
print("🎉 Tutorial Complete!")
print("\nYou now have a solid foundation in LangChain sequential chaining.")
print("Feel free to modify the code above and experiment with different:")
print("- Topics and prompts")
print("- Chain configurations")
print("- Error handling strategies")
print("- Performance optimizations")
print("\nHappy coding! 🚀")

#
# ## Bonus: Quick Reference
#
# ```python
# # Basic chain pattern
# chain = LLMChain(llm=llm, prompt=prompt)
# result = chain.run(input_variable="value")
#
# # Sequential chain pattern
# sequential_chain = SimpleSequentialChain(
#     chains=[chain1, chain2, chain3],
#     verbose=True
# )
# result = sequential_chain.run("input")
#
# # Multi-input sequential chain
# advanced_chain = SequentialChain(
#     chains=[chain1, chain2],
#     input_variables=["input1", "input2"],
#     output_variables=["output1", "output2"]
# )
# result = advanced_chain({"input1": "value1", "input2": "value2"})
# ```